# 情感分类

### 数据处理

In [1]:
import pandas as pd 
import numpy as np 
import jieba 

neg=pd.read_excel('neg.xls',header=None,index=None)
pos=pd.read_excel('pos.xls',header=None,index=None) 
pos['mark']=1
neg['mark']=0 

#合并语料
pn=pd.concat([pos,neg],ignore_index=True) 

#计算语料数目
neglen=len(neg)
poslen=len(pos) 

#定义分词函数
cw = lambda x: list(jieba.cut(x)) 
pn['words'] = pn[0].apply(cw)

#读入评论内容
comment = pd.read_excel('sum.xls') 
comment = comment[comment['rateContent'].notnull()] 
comment['words'] = comment['rateContent'].apply(cw) 

d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True) 

w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)

dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
del w,d2v_train
dict['id']=list(range(1,len(dict)+1))

get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢
print(pn['sent'])
print(pn['mark'])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5s/m09w_9qn6lzg3_08p9yy1h1m0000gn/T/jieba.cache
Loading model cost 0.882 seconds.
Prefix dict has been built succesfully.


0        [151, 521, 266, 45, 18, 14529, 97, 2, 1788, 1,...
1        [146, 4041, 19446, 2851, 2, 840, 1, 736, 793, ...
2        [146, 7758, 5470, 1046, 4556, 21529, 364, 23, ...
3        [146, 13, 20445, 9542, 405, 31, 4, 1648, 8891,...
4        [146, 13, 3181, 23386, 11668, 287, 1, 68, 2255...
                               ...                        
21100    [28, 8, 40, 394, 4, 1603, 69, 1, 209, 666, 389...
21101    [61, 19, 1, 223, 88, 181, 1, 28, 35, 131, 1540...
21102    [1482, 1176, 2, 24, 155, 2319, 2, 197, 1, 12, ...
21103            [213, 19, 1, 211, 11, 19, 1, 223, 181, 1]
21104    [209, 223, 88, 659, 1, 224, 223, 16, 5, 1820, ...
Name: sent, Length: 21105, dtype: object
0        1
1        1
2        1
3        1
4        1
        ..
21100    0
21101    0
21102    0
21103    0
21104    0
Name: mark, Length: 21105, dtype: int64


In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding
from tensorflow.keras.layers import LSTM, GRU

maxlen = 50

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))

x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

print(xa.shape)
print(ya.shape)

Pad sequences (samples x time)
(21105, 50)
(21105,)


In [8]:
model = Sequential()
model.add(Embedding(len(dict) + 1, 256))
model.add(LSTM(128, dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(x.shape)
print(y.shape)
batch_size = 32
model.summary()
model.fit(
    x=x,
    y=y,
    batch_size=batch_size,
    epochs=10,
    verbose=2,
    validation_data=(xt, yt)
)

(10553, 50)
(10553,)
53000
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 128)         6784128   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 6,915,841
Trainable params: 6,915,841
Non-trainable params: 0
_________________________________________________________________
Train on 10553 samples, validate on 10552 samples
Epoch 1/10
10553/10553 - 62s - loss: 0.4580 - acc: 0.7852 - val_loss: 0.3214 - val_acc: 0.8720
Epoch 2/10
10553/10553 - 61s - loss: 0.2029 - acc: 0.9272 - val_loss: 0.3076 - val_acc: 0.8879
Epoch 3/10


KeyboardInterrupt: 